In [17]:
import pandas as pd
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow import lite

In [18]:
df=pd.read_excel('GameScore_dataset.xlsx')
properties = list(df.columns.values)
properties.remove('alzheimer')
X = df[properties]
y = df['alzheimer']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [20]:
model = keras.Sequential([
    keras.layers.Dense(9,input_shape=[9,]),
    #keras.layers.Flatten(input_shape=[9,]),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [21]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=50, batch_size=1)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/50
198/198 [==============================] - 1s 3ms/step - loss: 0.7391 - accuracy: 0.8384
Epoch 2/50
198/198 [==============================] - 0s 2ms/step - loss: 0.4568 - accuracy: 0.8838
Epoch 3/50
198/198 [==============================] - 1s 3ms/step - loss: 0.3811 - accuracy: 0.8889
Epoch 4/50
198/198 [==============================] - 1s 3ms/step - loss: 0.3659 - accuracy: 0.8990
Epoch 5/50
198/198 [==============================] - 1s 3ms/step - loss: 0.3591 - accuracy: 0.8939
Epoch 6/50
198/198 [==============================] - 1s 3ms/step - loss: 0.3072 - accuracy: 0.8990
Epoch 7/50
198/198 [==============================] - 1s 3ms/step - loss: 0.3618 - accuracy: 0.9040
Epoch 8/50
198/198 [==============================] - 1s 4ms/step - loss: 0.3575 - accuracy: 0.8990
Epoch 9/50
198/198 [==============================] - 0s 2ms/step - loss: 0.3007 - accuracy: 0.9141 ETA: 0s - loss: 0.2227 - accu
Epoch 10/50
198/198 [==============================] - 0s 2ms/step - l

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_6 (Dense)              (None, 16)                160       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 539
Trainable params: 539
Non-trainable params: 0
_________________________________________________________________


In [24]:
X_test[1:2]['speed']

61    4
Name: speed, dtype: int64

In [25]:
y_pred=model.predict(X_test)

In [26]:
print(int(a))

NameError: name 'a' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
NN_mse=mean_squared_error(y_test,y_pred)
print(NN_mse)

In [ ]:
rms_NN = sqrt(mean_squared_error(y_test, y_predicted))
print(rms_NN)

In [ ]:
#saving model by using tf save model
model.save('saved_model') 

In [ ]:
#saving model as pkl
import pickle
weigh= model.get_weights()    
pklfile= "modelweights.pkl"

fpkl= open(pklfile, 'wb')    #Python 3     

pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
print(type(pklfile))
fpkl.close()

In [ ]:
keras_file="linear.h5"
keras.models.save_model(model,keras_file)

In [ ]:
converter= lite.TocoConverter.from_keras_model_file(keras_file)
tflite_model=converter.convert()
open("NN_game_score.tflite", "wb").write(tflite_model)

In [ ]:
model.evaluate_